In [1]:
pip install xlrd

In [5]:
conda install openpyxl

Channels:
 - defaults
Platform: win-64
Solving environment: ...working... done

# All requested packages already installed.


Note: you may need to restart the kernel to use updated packages.


In [2]:
pip install inquirer

Note: you may need to restart the kernel to use updated packages.


In [2]:
# import pandas lib as pd
import pandas as pd

In [3]:
import numpy as np

In [4]:
import math

## I. Read input files

In [6]:
# open key tables from excel file
supermarket_catalogue = pd.read_excel(r"C:\Users\Louis\Box\Personal\Python\pandas\Recipes\Food data categories.xlsx",sheet_name="Supermarket catalogue")
supermarket_catalogue = supermarket_catalogue.astype({'ingredient id': int,
                                                      'ingredient': str,
                                                      'quantity': float,
                                                      'unit': str,
                                                      'price (£)':float})
                                                      
recipes = pd.read_excel(r"C:\Users\Louis\Box\Personal\Python\pandas\Recipes\Food data categories.xlsx",sheet_name="Recipes")
recipes = recipes.astype({'recipe id': int,
                           'recipe': str,
                          '#people':int})

ingredient_list = pd.read_excel(r"C:\Users\Louis\Box\Personal\Python\pandas\Recipes\Food data categories.xlsx",sheet_name="Ingredient list")
ingredient_list = ingredient_list.astype({'recipe id': int,
                                          'recipe': str,
                                          'ingredient': str,
                                         'ingredient id':int,
                                          'quantity': float,
                                          'unit': str})

In [7]:
supermarket_catalogue.head()

,ingredient id,ingredient,quantity,unit,price (£)
0,1,Butter,250.0,g,2.0
1,2,Milk,1000.0,L,1.0
2,3,Sugar,500.0,g,1.0
3,4,Flour,1000.0,g,1.0
4,5,Eggs,6.0,unit,2.5


In [8]:
recipes.head()

,recipe id,recipe,#people
0,1,Chicken mushroom rice,2
1,2,Carbonara,2
2,3,Arrabiata,1
3,4,Steak & fries,1
4,5,Chicken nuggets,4


In [9]:
ingredient_list

,recipe id,recipe,ingredient,ingredient id,quantity,unit
0,1,Chicken mushroom rice,Chicken Thigh,8,300.00,g
1,1,Chicken mushroom rice,Mushrooms,27,200.00,g
2,1,Chicken mushroom rice,Rice,12,100.00,g
3,1,Chicken mushroom rice,Cream,28,100.00,ml
4,1,Chicken mushroom rice,Salt,35,20.00,g
5,1,Chicken mushroom rice,Pepper,36,20.00,g
6,2,Carbonara,Eggs,5,2.00,unit
7,2,Carbonara,Parmesan,29,50.00,g
8,2,Carbonara,Guanciale,10,75.00,g
9,2,Carbonara,Pasta,11,200.00,g


## II. Calculate recipe prices

In [10]:
#Create blank columns in the recipes table
recipes["recipe_order_price (£)"]=" "
recipes["recipe_cooking_price (£)"]=" "
recipes["recipe_cooking_price_pp (£)"]=" "
sum_order_price = 0
sum_cooking_price = 0
sum_cooking_price_pp = 0

#Loop on all recipes
for index, rec in recipes.iterrows():
    
    i=rec['recipe id']
    
    #Within a recipe loop on all its ingredients
    for index, row in ingredient_list[ingredient_list['recipe id']==i].iterrows():    
        
        #calculate 1) the cooking quantity (the amount of ingredient you need for the recipe), 
        #2) the catalogue quantity (the minimum amount you need to buy from the supermarket)
        # 3) the cooking multiple (how many units of catalogue ingredients are needed to cook the recipe)
        # 4) the order multiple (how many units of catalogue ingredients to purchase)
        ingredient_id=row['ingredient id']
        cooking_quantity=row['quantity']
        catalogue_quantity=supermarket_catalogue[supermarket_catalogue['ingredient id']==ingredient_id]['quantity'].item()
        cooking_multiple=cooking_quantity/catalogue_quantity
        order_multiple=math.ceil(cooking_quantity/catalogue_quantity)
        
        #sums prices of each ingredient but only the portion used in the recipe
        sum_cooking_price+=supermarket_catalogue[supermarket_catalogue['ingredient id']==ingredient_id]['price (£)'].item()*cooking_multiple
       
        #sums prices of each bought ingredient, bought entirely
        sum_order_price+=supermarket_catalogue[supermarket_catalogue['ingredient id']==ingredient_id]['price (£)'].item()*order_multiple
    
    #calculates recipe's cooking price per person
    cooking_price_pp=sum_cooking_price/rec['#people']

    #writes calculations into the recipe table
    recipes.loc[recipes['recipe id']==i,"recipe_cooking_price (£)"]=round(sum_cooking_price,2)
    recipes.loc[recipes['recipe id']==i,"recipe_cooking_price_pp (£)"]=round(cooking_price_pp,2)
    recipes.loc[recipes['recipe id']==i,"recipe_order_price (£)"]=round(sum_order_price,2)

    sum_order_price = 0
    sum_cooking_price = 0
    
recipes.head()

,recipe id,recipe,#people,recipe_order_price (£),recipe_cooking_price (£),recipe_cooking_price_pp (£)
0,1,Chicken mushroom rice,2,16.5,6.8,3.4
1,2,Carbonara,2,17.0,5.02,2.51
2,3,Arrabiata,1,21.35,4.65,4.65
3,4,Steak & fries,1,14.0,6.02,6.02
4,5,Chicken nuggets,4,20.5,11.98,3.0


## III. Ask user to chose recipes 

In [11]:
>>> import operator
recipe_list = pd.DataFrame(recipes['recipe'][:])

In [12]:
def choose_index():
    return int(input())

In [13]:
#Function to allow the user to select recipes from an available list
def Select_Recipes(list):
    
    Chosen_recipes = []
    index = ""

    print (list)
    
    print("\n To add a recipe, please type the index number. Once you have selected all your recipes, please type 999 \n")
    index = choose_index()

    while index != 999:
        Chosen_recipes.append(index)
        index = choose_index()

    return Chosen_recipes

In [14]:
#Asks the user to select recipes from the available list
Chosen_recipes_ids =[]
Chosen_recipes_indexes = Select_Recipes(recipe_list)
Chosen_recipes_ids = [x+1 for x in Chosen_recipes_indexes]
print("Recipes chosen are ", operator.itemgetter(*Chosen_recipes_indexes)(recipe_list["recipe"]))

                  recipe
0  Chicken mushroom rice
1              Carbonara
2              Arrabiata
3          Steak & fries
4        Chicken nuggets

 To add a recipe, please type the index number. Once you have selected all your recipes, please type 999 

0
1
2
4
999
Recipes chosen are  ('Chicken mushroom rice', 'Carbonara', 'Arrabiata', 'Chicken nuggets')


## IV. Summarise order quantities and cost

In [15]:
#filter ingredient dataset for the chosen recipes
shopping_list = ingredient_list.loc[ingredient_list['recipe id'].isin(Chosen_recipes_ids)]

#sums the quantity together
shopping_list= pd.DataFrame(shopping_list.groupby('ingredient id',as_index=False)['quantity'].sum())
shopping_list=shopping_list.rename(columns={"quantity":"cooking quantity"})

#Adds back the other ingredient columns like ingredient name and unit price
shopping_list = pd.merge(shopping_list, supermarket_catalogue.rename(
    columns={"quantity": "unit supermarket quantity","price (£)":"unit price (£)"}), how="left", on="ingredient id")

#reorders columns
shopping_list = shopping_list[['ingredient id','ingredient','unit','unit supermarket quantity','unit price (£)','cooking quantity']]

#adds cooking price as cooking quantity * unit price in supermarket
shopping_list['cooking price (£)']=shopping_list['cooking quantity']/shopping_list['unit supermarket quantity']*(
    shopping_list['unit price (£)'])
shopping_list['cooking price (£)']=round(shopping_list['cooking price (£)'],2)

#adds order quantity
df=shopping_list['cooking quantity']/shopping_list['unit supermarket quantity']
shopping_list['order quantity (units)']=df.apply(np.ceil)

#adds order price as order quantity * unit price in supermarket
shopping_list['order price (£)']=shopping_list['order quantity (units)']*shopping_list['unit price (£)']
shopping_list['order price (£)']=round(shopping_list['order price (£)'],2)

shopping_list

,ingredient id,ingredient,unit,unit supermarket quantity,unit price (£),cooking quantity,cooking price (£),order quantity (units),order price (£)
0,4,Flour,g,1000.0,1.00,200.00,0.20,1.0,1.00
1,5,Eggs,unit,6.0,2.50,5.00,2.08,1.0,2.50
2,8,Chicken thigh,g,500.0,6.00,900.00,10.80,2.0,12.00
3,10,Guanciale,g,150.0,4.00,75.00,2.00,1.0,4.00
4,11,Pasta,g,500.0,2.00,400.00,1.60,1.0,2.00
5,12,Rice,g,1000.0,1.50,100.00,0.15,1.0,1.50
6,15,Tomatoes,unit,1.0,0.70,2.00,1.40,2.0,1.40
7,21,Basil,bag,1.0,1.95,0.50,0.98,1.0,1.95
8,23,Garlic,unit,1.0,0.50,0.25,0.12,1.0,0.50
9,27,Mushrooms,g,200.0,2.00,200.00,2.00,1.0,2.00


In [16]:
Total_order_price=shopping_list['order price (£)'].sum()
print ("Your total basket is worth", Total_order_price, "euros")

Your total basket is worth 51.35 euros
